In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('fake_news.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.shape

(20800, 5)

In [4]:
df.isnull().mean()*100

id        0.000000
title     2.682692
author    9.408654
text      0.187500
label     0.000000
dtype: float64

In [5]:
df.dropna(inplace=True)

In [6]:
X = df.drop('label', axis="columns")
y = df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential

In [9]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

### Text Preprocessing

In [11]:
messages = X.copy()

In [12]:
len(messages)

18285

In [13]:
messages.reset_index(inplace=True)

In [14]:
messages.title

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                        Why the Truth Might Get You Fired
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
18280    Rapper T.I.: Trump a ’Poster Child For White S...
18281    N.F.L. Playoffs: Schedule, Matchups and Odds -...
18282    Macy’s Is Said to Receive Takeover Approach by...
18283    NATO, Russia To Hold Parallel Exercises In Bal...
18284                            What Keeps the F-35 Alive
Name: title, Length: 18285, dtype: object

In [15]:
messages.head(8)

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
6,7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
7,9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...


In [16]:
porter = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    sentence = re.sub("[^a-zA-z]", ' ', messages['title'][i])
    sentence = sentence.lower()
    sentence = sentence.split()
    
    sentence = [porter.stem(word) for word in sentence if not word in stopwords.words('english')]
    sentence = ' '.join(sentence)
    corpus.append(sentence)

In [18]:
voc_size=5000

In [19]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 

### Embedding Representation

In [20]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2664  802 1460]
 [   0    0    0 ... 3367 4088 1359]
 [   0    0    0 ... 3351 3639 3835]
 ...
 [   0    0    0 ... 4893 2267 3055]
 [   0    0    0 ... 3782 4806 2657]
 [   0    0    0 ...  407  562 2857]]


In [21]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1939,
       2639, 3996,  970, 4799, 4358, 3298, 2664,  802, 1460])

In [22]:
sent_length

20

In [23]:
voc_size

5000

In [25]:
from tensorflow.keras.layers import Dense

In [40]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [41]:
len(embedded_docs),y.shape

(18285, (18285,))

In [42]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [44]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 1s 7ms/step - loss: 0.3370 - accuracy: 0.8459 - val_loss: 0.2017 - val_accuracy: 0.9157
Epoch 2/10
192/192 [==============================] - 1s 5ms/step - loss: 0.1379 - accuracy: 0.9466 - val_loss: 0.2046 - val_accuracy: 0.9246
Epoch 3/10
192/192 [==============================] - 1s 5ms/step - loss: 0.0980 - accuracy: 0.9639 - val_loss: 0.2131 - val_accuracy: 0.9183
Epoch 4/10
192/192 [==============================] - 1s 5ms/step - loss: 0.0644 - accuracy: 0.9778 - val_loss: 0.2555 - val_accuracy: 0.9167
Epoch 5/10
192/192 [==============================] - 1s 5ms/step - loss: 0.0373 - accuracy: 0.9879 - val_loss: 0.2986 - val_accuracy: 0.9150
Epoch 6/10
192/192 [==============================] - 1s 5ms/step - loss: 0.0252 - accuracy: 0.9917 - val_loss: 0.4074 - val_accuracy: 0.9082
Epoch 7/10
192/192 [==============================] - 1s 5ms/step - loss: 0.0162 - accuracy: 0.9952 - val_loss: 0.4505 - val_accuracy: 0.9130
Epoch 

In [31]:
# from tensorflow.keras.layers import Dropout
# ## Creating model
# embedding_vector_features=40
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(Dropout(0.3))
# model.add(LSTM(100))
# model.add(Dropout(0.3))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [45]:
y_pred=model.predict_classes(X_test)

In [46]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [47]:
from sklearn.metrics import confusion_matrix

In [48]:
confusion_matrix(y_test,y_pred)

array([[3099,  320],
       [ 221, 2395]], dtype=int64)

In [49]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9103562551781276